In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('EURUSD_data')

# Select features and target
features = data[['Open', 'High', 'Low', 'Close', 'Adj Close']]
target = data['Close']

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(features)

# Create sequences of data for LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length, 3])  # Use the 'Close' price as target
    return np.array(X), np.array(y)

seq_length = 60  # Using 60 days of data to predict the next day's price
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

### Step 1: Define the Evaluation Function

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from geneticalgorithm import geneticalgorithm as ga
import numpy as np
from tensorflow.keras.optimizers.legacy import Adam

# Define the evaluation function
def evaluate_lstm(params):
    units_layer_1 = int(params[0])
    units_layer_2 = int(params[1])
    dropout_rate = float(params[2])
    learning_rate = float(params[3])
    
    # Initialize the model
    model = Sequential()
    
    # LSTM layer 1
    model.add(LSTM(units=units_layer_1, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout_rate))
    
    # LSTM layer 2
    model.add(LSTM(units=units_layer_2, return_sequences=False))
    model.add(Dropout(dropout_rate))
    
    # Output layer
    model.add(Dense(units=1))
    
    # Use the legacy Adam optimizer
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)  # Using fewer epochs for quicker evaluation
    
    # Predict on test data
    predicted_prices = model.predict(X_test)
    
    # Inverse transform the predicted and actual values to get them in the original scale
    predicted_prices = scaler.inverse_transform(np.concatenate([X_test[:, -1, :-1], predicted_prices], axis=1))[:, -1]
    actual_prices = scaler.inverse_transform(np.concatenate([X_test[:, -1, :-1], y_test.reshape(-1, 1)], axis=1))[:, -1]
    
    # Evaluate the model
    mse = np.mean((predicted_prices - actual_prices) ** 2)
    
    return mse  # The GA will minimize this value

### Step 2: Define the Genetic Algorithm Configuration

In [8]:
# Define the variable bounds
varbound = np.array([
    [50, 200],  # units_layer_1: Range for units in the first LSTM layer
    [20, 100],  # units_layer_2: Range for units in the second LSTM layer
    [0.1, 0.5],  # dropout_rate: Range for dropout rate
    [0.0001, 0.01]  # learning_rate: Range for learning rate
])

# Initialize the Genetic Algorithm
modelGA = ga(function=evaluate_lstm, dimension=4, variable_type='real', variable_boundaries=varbound, function_timeout=60)  # Increase timeout to 60 seconds

# Run the Genetic Algorithm
modelGA.run()

# Best parameters found by GA
best_params = modelGA.output_dict['variable']
best_mse = modelGA.output_dict['function']

print('Best parameters found by genetic algorithm:', best_params)
print("Best Mean Squared Error:", best_mse)

16/16 [==============================] - 3s 43ms/step
given function is not applicable


AssertionError: After 60.0 seconds delay func_timeout: the given function does not provide any output